# NBA Standings

In [71]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import numpy as np

years = range(1989,2020,1)
teams = [u'Vancouver Grizzlies','Charlotte Bobcats','Seattle Supersonics','New Orleans Hornets',u'Toronto Raptors', u'Brooklyn Nets', u'Boston Celtics', u'New York Knicks', u'Philadelphia 76ers', u'Chicago Bulls', u'Cleveland Cavaliers', u'Milwaukee Bucks', u'Indiana Pacers', u'Detroit Pistons', u'Atlanta Hawks', u'Washington Wizards', u'Miami Heat', u'Orlando Magic', u'Charlotte Hornets', u'Portland Trail Blazers', u'Oklahoma City Thunder', u'Denver Nuggets', u'Utah Jazz', u'Minnesota Timberwolves', u'Golden State Warriors', u'Los Angeles Clippers', u'Phoenix Suns', u'Sacramento Kings', u'Los Angeles Lakers', u'Memphis Grizzlies', u'Houston Rockets', u'Dallas Mavericks', u'San Antonio Spurs', u'New Orleans Pelicans']
base_url = "https://www.basketball-reference.com/leagues/NBA_{}.html"
nba = pd.DataFrame()


for year in years:
    url = base_url.format(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for team in teams:
        try:
            wins = soup.find(text = re.compile(team)).find_next("td").string
            losses = soup.find(text = re.compile(team)).find_next("td").find_next("td").string
            df = pd.DataFrame([team,wins,losses]).T
            df['Year'] = year
            nba = nba.append(df)
        except:
            pass
        
nba.columns = ['Team','Wins','Losses','Year']
nba = nba.reset_index()
nba.drop('index',axis = 1,inplace= True)
none_nba = nba[nba['Wins'].isnull()]
none_nba['Wins'] = none_nba['Losses']
none_nba['Wins'] = none_nba['Wins'].astype('int32')
none_nba['Losses'] = none_nba['Losses'].astype('int32')
none_nba['Losses'] = 82 - none_nba['Wins']


nba.drop(none_nba.index,inplace = True)
nba = nba.append(none_nba)
nba['Wins'] = nba['Wins'].astype('int32')
nba['Losses'] = nba['Losses'].astype('int32')
nba['win%'] = nba['Wins']/(nba['Wins'] + nba['Losses'])

#removing 1999 and 2012 seasons as they were lockout shortened to 50 and 66 games 
nba_fs = nba[(nba['Year'] != 2012) & (nba['Year'] != 1999)]


(nba_fs['win%'].std())

# NHL Standings

In [ ]:
years = np.arange(1989,2020,1)
base_url = 'https://www.hockey-reference.com/leagues/NHL_{}_standings.html'

nhl = pd.DataFrame()

for year in years:
    if year == 2005: #2005 saw no games played due to lockout 
        pass
    else:
        req_url = base_url.format(year)
        req = requests.get(req_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find('table', {'id':'standings'})
        df = pd.read_html(str(table))[0]
        df['Year'] = year
        nhl = nhl.append(df[['Unnamed: 1','Overall','Year']])

nhl.columns = ['Team','Overall','Year']
nhl = nhl[nhl['Year'] != 2013] #removing 2013 season as it was lockout shortened
record_nhl = nhl.Overall.str.split("-",expand = True)
record_nhl.columns = ['Wins','RegL','OTL']

record_nhl['RegL'] = record_nhl['RegL'].astype('int32')
record_nhl['OTL'] = record_nhl['OTL'].astype('int32')
record_nhl['Wins'] = record_nhl['Wins'].astype('int32')
record_nhl['Losses'] = record_nhl['RegL'] + record_nhl['OTL']
record_nhl['win%'] = record_nhl['Wins']/(record_nhl['Wins'] + record_nhl['Losses'])

In [96]:
nhlvarob = (record_nhl['win%'].std())**2

0.003048780487804878

In [1]:
nbavarob = nba['win%'].std()**2

NameError: name 'nba' is not defined

In [97]:
nba['win%'].std()**2

0.023426033393787064

In [93]:
.19**2

0.0361

In [109]:
url = 'https://www.basketball-reference.com/playoffs/NBA_2019.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
wins = soup.find_all(id="s874")

In [106]:
url = 'https://www.basketball-reference.com/leagues/NBA_2019.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
wins = soup.find(text = re.compile('Toronto Raptors')).find_next("td").string

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import numpy as np

finals_url = {}
years = range(1989,2020,1)
i = 0


for year in years:
    url = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
    page = requests.get(url.format(year))
    soup = BeautifulSoup(page.content, 'html.parser')  
    for a in soup.find_all('a', href=True):
        if 'nba-finals-' in a['href']:
            i += 1
            finals_url[i] = a['href']
        else:
            pass

In [2]:
allurls = list(set(finals_url.values()))
allurls.sort()

In [ ]:
url = '/playoffs/1990-nba-finals-trail-blazers-vs-pistons.html'
nba_finals_stats = pd.DataFrame()
bballrefurl = 'https://www.basketball-reference.com{}'
# i = 1
year = 1989

# for url in allurls:

    
def finals_stats(url):
    year = allurls[allurls.index(url)][10:14]
#     i = 1
    print(year)
    finalsurl = bballrefurl.format(url)
    print(finalsurl)
    df = pd.DataFrame()  
    for n in [-1,-2]:
        t1 = pd.read_html(finalsurl)[n]
        t1.columns = t1.columns.get_level_values(1) 
        t1['Team'] = 'Team' + str(n*-1)
        t1['Year'] = year
        df = df.append(t1[['Rk','Player','MP','Team','Year']])
#         print(df)
    return df

for url in allurls:
    nba_finals_stats = nba_finals_stats.append(finals_stats(url))
    

In [44]:
nba_finals_stats.to_csv('nbafinalsstats.csv')


In [17]:
t1 = pd.read_html('https://www.basketball-reference.com/playoffs/1989-nba-finals-lakers-vs-pistons.html')[8]
i += 1
# print(t1)
t1.columns = t1.columns.get_level_values(1) 
# t1['Team'] = 'Team' + str(i)
# t1['Year'] = year
nba_finals_stats = nba_finals_stats.append(t1[['Rk','Player']])
nba_finals_stats

,Rk,Player
0,1.0,Joe Dumars
1,2.0,Isiah Thomas
2,3.0,Vinnie Johnson
3,4.0,James Edwards
4,5.0,Bill Laimbeer
5,6.0,Mark Aguirre
6,7.0,John Salley
7,8.0,Rick Mahorn
8,9.0,Dennis Rodman
9,10.0,John Long


In [52]:
url = 'https://www.hockey-reference.com/playoffs/NHL_2018.html'
page = requests.get(url.format(year))
soup = BeautifulSoup(page.content, 'html.parser')  

for a in soup.find_all('a', href=True):
    if 'stanley' in a['href']:
        print(a['href'])

/playoffs/2018-vegas-golden-knights-vs-washington-capitals-stanley-cup-final.html


In [93]:
nhlfinals_url = 
years = range(1989,2020,1)
i = 0


for year in years:
    url = 'https://www.hockey-reference.com/playoffs/NHL_{}.html'
    page = requests.get(url.format(year))
    soup = BeautifulSoup(page.content, 'html.parser')  
    for a in soup.find_all('a', href=True):
        if 'stanley' in a['href']:
            i += 1
            nhlfinals_url[i] = a['href']
        else:
            pass

In [95]:
allnhlurls = list(set(nhlfinals_url.values()))
allnhlurls.sort()

In [ ]:
nhlrefurl = 'https://www.hockey-reference.com{}'
nhl_finals_stats = pd.DataFrame()

def nhlfinals_stats(url):
    print(url)
    year = allnhlurls[allnhlurls.index(url)][10:14]
#     i = 1
    print(year)
    finalsurl = nhlrefurl.format(url)
    print(finalsurl)
    df = pd.DataFrame()  
    for n in [-2,-4]:
        t1 = pd.read_html(finalsurl)[n]
        t1.columns = t1.columns.get_level_values(1) 
        t1['Team'] = 'Team' + str(int(n*-.5))
        t1['Year'] = year
        df = df.append(t1[['Rk','Player','Pos','ATOI','Team','Year']])
#         print(df)
    return df

for url in allnhlurls:
    nhl_finals_stats = nhl_finals_stats.append(nhlfinals_stats(url))


In [ ]:
nhl_finals_stats.to_csv('nhlfinalsstats.csv')

In [1]:
# variance in winning% between nba and nhl over the 30 years 
# plot variance over the years
# plot playing time% in finals as well
# calculate PS/WS divided by league average to show difference in contribution between stars and 
# trend persists over the years; nhl win% variance potentially higher prior to 2005
# star players playing more minutes having more influence; 80% for NBA vs 33% for NHL 
# star players have roughly 4.5x win shares/point shares compared to league average

# talk about repeats pre and post NHL hard cap which was introduced in 2006 after 2005 lockout
# hard cap vs soft cap allowing continued success which is a major difference in the collective bargaining agreements 
# of the 2 leagues 




### take a look with a plot 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

